# Tugas 3 (Regression Time Series Data Passenger)

## Get and prepare Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
df_data.head(10)

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
5,1949-06,135
6,1949-07,148
7,1949-08,148
8,1949-09,136
9,1949-10,119


In [ ]:
df_data.shape

(144, 2)

In [ ]:
df_passenger= df_data['Passengers']

In [ ]:
# transform univariate time series to supervised learning problem
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        # print(i, end_ix)
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## tuning data

In [ ]:
n_steps = 13
X, y = split_sequence(df_passenger, n_steps)

In [ ]:
# column names to X and y data frames
df_X = pd.DataFrame(X, columns=['t-'+str(i) for i in range(n_steps-1, -1,-1)])
df_y = pd.DataFrame(y, columns=['t+1 (prediction)'])

# concat df_X and df_y
df = pd.concat([df_X, df_y], axis=1)

In [ ]:
# df_X.head()
# df_y.head()
df.head(3)

,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0,t+1 (prediction)
0,112,118,132,129,121,135,148,148,136,119,104,118,115,126
1,118,132,129,121,135,148,148,136,119,104,118,115,126,141
2,132,129,121,135,148,148,136,119,104,118,115,126,141,135


## prepo

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
X_norm= scaler.fit_transform(df_X)
# y_norm= scaler.fit_transform(df_y)

In [ ]:
X_norm

array([[0.01758242, 0.03076923, 0.06153846, ..., 0.        , 0.00787402,
        0.0019685 ],
       [0.03076923, 0.06153846, 0.05494505, ..., 0.02702703, 0.0019685 ,
        0.02362205],
       [0.06153846, 0.05494505, 0.03736264, ..., 0.02123552, 0.02362205,
        0.05314961],
       ...,
       [0.78901099, 0.66593407, 0.56703297, ..., 1.        , 0.96850394,
        0.77559055],
       [0.66593407, 0.56703297, 0.66153846, ..., 0.96911197, 0.77559055,
        0.68307087],
       [0.56703297, 0.66153846, 0.68791209, ..., 0.77992278, 0.68307087,
        0.54330709]])

## Split data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=0)

## model

In [ ]:
# import knn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

model_knn = KNeighborsRegressor(n_neighbors=4)


## fit and predict knn

In [ ]:
model_knn.fit(X_train, y_train)
y_pred=model_knn.predict(X_test)

## evaluasi

dimana nilai r2 score semakin mendekati 1 semakin baik atau model yang dihasilkan  memiliki prediksi mendekati real dan sebaliknya semakin mendekati 0 semakin buruk atau model yang dihasilkan tidak mendekati real biasanya di berikan threshold 0.5.
<!-- latex r2 score -->
$$SS_{tot}=\sum_{i=1}^{n}(y^i-\bar{y})^2$$
$$SS_{res}=\sum_{i=1}^{n}(y^i-f(X^i))^2$$
$$R^2=1-\frac{SS_{res}}{SS_{tot}}$$

In [ ]:
print('R-Squared:', model_knn.score(X_test, y_test))

R-Squared: 0.9614748441693041


dimana nilai mse semakin kecil semakin baik atau model yang dihasilkan memiliki prediksi mendekati real.
<!-- latex mse -->

$$ \text{MSE} = \frac{1}{n} \sum_{i=0}^n (y_i - \hat{y}_i)^2$$ 
$$MSE=\frac{1}{n}\sum_{i=1}^{n}(y^i-f(X^i))^2$$


In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

372.43055555555554

## result

In [ ]:
y_test.shape

(27,)

In [ ]:
y_pred.shape

(27,)

In [ ]:
df_y_test = pd.DataFrame(y_test,columns=['y_test'])
df_y_pred = pd.DataFrame(y_pred,columns=['y_pred'])

df_hasil = pd.concat([df_y_test, df_y_pred], axis=1)

In [ ]:
df_hasil

,y_test,y_pred
0,133,144.75
1,267,244.50
2,467,444.50
3,337,360.25
4,284,276.75
5,242,212.00
6,211,212.25
7,172,181.75
8,233,217.25
9,181,206.00


In [ ]:
# df_hasil.to_excel('df_hasil n_step={}.xlsx'.format(n_steps), index=False)